# Deep SARSA (Frozen Lake)
---
## 환경
* state : discrete[16]
* action : discrete[4]
* 목표에 도달 시 +1 점, 구멍에 빠질 때 -1 점, 나머지 0 점.
---
* input : state (array[16])
* output : Q (prob)
---
* 상태를 바꿔넣는데 신경망 출력값이 동일하게 나오는 현상? - 마이너스 보상을 피하려고 맵밖으로 가는 행동때문
* 스텝마다 부정 보상을 주면 탈출하려는 행동만 한다. 어쩔 수 없는듯
* 환경이 좀 별로다. 긍정보상을 받아야 학습이 되는데 긍정보상 받을 확률조차 낮다.

In [6]:
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
import numpy as np
import gym
from gym.envs.registration import register
import random

In [7]:
'''
환경셋팅 한 후에 환경을 추가등록한다.
'''

register(
    id='FrozenLake-v1',
    entry_point="gym.envs.toy_text:FrozenLakeEnv",
    kwargs={'map_name':'4x4','is_slippery':False})

Error: Cannot re-register id: FrozenLake-v1

In [8]:
'''
환경 생성
'''
env = gym.make('FrozenLake-v1')

In [9]:
n_1 = 24
n_2 = 24

In [10]:
# 신경망 모델 - Q 값 근사
# state(array(16)) --> [NN] --> Q(array[4])
# output 은 예측값.
q = Sequential()
q.add(Dense(n_1, input_dim=env.observation_space.n, activation='sigmoid'))
q.add(Dense(n_2, activation='sigmoid'))
q.add(Dense(env.action_space.n, activation='softmax'))
q.compile(loss='mse', optimizer=Adam(lr=.1))

print(env.observation_space.n)
print(env.action_space.n)

16
4


In [30]:
gamma = .95
epsilon = 1
fanalty_step = .01
episode = 0
max_episode = 2000

state = env.reset()
action = env.action_space.sample()
step = 0

# state 가공
state = np.eye(env.observation_space.n)[state]
state = np.reshape(state, [1, env.observation_space.n])

while(episode < max_episode):
    
    step += 1
    
    # action 선택
    if(random.random() > epsilon):
        pi = q.predict(state)
        #print(pi)
        action = np.argmax(pi)
    else:
        action = env.action_space.sample()
        
    # 1 step
    state_next, reward, done, _ = env.step(action)
    
    # 시간 패널티
    reward -= fanalty_step * step
    
    # state 가공
    state_next = np.eye(env.observation_space.n)[state_next]
    state_next = np.reshape(state_next, [1, env.observation_space.n])
    
    # 임시 target Q
    q_target = np.zeros(4)
    
    # episode 종료 시
    if(done):
        
        # 타겟 계산
        q_target = reward + gamma * q.predict(state_next)

        # 구멍에 빠질 때 부정적 보상
        if(reward > 0):
            print(reward)
            #print(q_target)

            # epsilon 조절
            if(epsilon > .1):
                epsilon -= .05
            else:
                epsilon = .1
                
        else:
            q_target[0][action] = -2
        
        # 타겟과 예측값의 오차를 이용해 학습. 매 step 마다 target 에 가까워지게 업데이트
        q.fit(state, q_target, verbose=0)
        
        # episode 처리 - reset 등..
        step = 0
        episode += 1
        state = env.reset()
        
        # 새로운 state 가공
        state = np.eye(env.observation_space.n)[state]
        state = np.reshape(state, [1, env.observation_space.n])
        
    else:
        # 타겟 계산
        q_target = reward + gamma * q.predict(state_next)

        # 타겟과 예측값의 오차를 이용해 학습. 매 step 마다 target 에 가까워지게 업데이트
        q.fit(state, q_target, verbose=0)

        # state 업데이트
        state = state_next
        
env.close()

0.83
0.79
0.89
0.84
0.75
0.7
0.84
0.8200000000000001
0.81


In [31]:
s = env.reset()

step = 1
total_r = 0

while(True):
    #env.render()
    s = np.eye(env.observation_space.n)[s]
    s = np.reshape(s, [1, env.observation_space.n])
    a = q.predict(s)
    #print(a)
    #print(s)
    a = np.argmax(a)
    s1, r, d, _ = env.step(a)
    s = s1
    
    if(d):
        env.render()
        break
env.close()

KeyboardInterrupt: 